In [1]:
import dask.dataframe as ddf
import pandas as pd
import numpy as np
import re
import os
import time
from glob import glob
from wikiwho_wrapper import WikiWho
import mwparserfromhell
import string
import requests
import math

In [2]:
ww = WikiWho('wikiProject','Apple1234')

In [4]:
parquetdir = '../../tcm-columns-add-main'

In [5]:
page_title = ddf.read_parquet(os.path.join(parquetdir,'page.title')).compute()
page_title = page_title.drop(columns={'dir0'})

In [6]:
timestamp = ddf.read_parquet(os.path.join(parquetdir,'revision.timestamp')).compute()
timestamp = timestamp.drop(columns={'dir0'})

In [7]:
df = page_title.join(timestamp)
df_max_time = df.groupby('page.title').agg({'revision.timestamp':'max'}).reset_index()
x = df_max_time.merge(df.reset_index(),on=['page.title','revision.timestamp'])
x = x.set_index('page.title')
x = x[~x.index.duplicated(keep='first')]

In [8]:
contributor = ddf.read_parquet(os.path.join(parquetdir,'contributor.username')).compute()
contributor = contributor.drop(columns={'dir0'})

In [9]:
contributor_id = ddf.read_parquet(os.path.join(parquetdir,'contributor.id')).compute()
contributor_id = contributor_id.drop(columns={'dir0'})

In [10]:
contributor = contributor.join(contributor_id)

In [11]:
contributor_info = contributor[~contributor['contributor.username'].duplicated(keep='first')].set_index('contributor.id')

In [12]:
contributor_info = contributor_info.to_dict()['contributor.username']

In [13]:
kept_users = pd.read_parquet('../../intermediate-result/TCM/editors-with-sig-contrib-at-least-10')
kept_users_list = kept_users.index

In [14]:
fin_df = contributor.join(page_title)

In [16]:
sr = {}
hl = {}

In [17]:
# x.index remove talks
all_titles = list(x.index)

In [18]:
all_articles = [i for i in all_titles if not i.startswith('Talk:')]

In [19]:
%%time
for title in all_articles:
    # sr
    # get content for most current content
    max_rev = x.loc[title]['index']
#     response = ww.api.specific_rev_content_by_rev_id(max_rev,title)
    try:
        response = ww.api.specific_rev_content_by_rev_id(max_rev,title)
    except requests.exceptions.RequestException as e:
        print(e)
        continue
    all_tokens = response['revisions'][0][max_rev]['tokens']
    kept_editor_count = {}
    for i,v in enumerate(d for d in all_tokens): 
        editor = v['editor']
        if editor not in contributor_info:
            continue
        editor_name = contributor_info[editor]
        if editor_name not in kept_users_list:
            continue
        if editor_name in kept_editor_count:
            kept_editor_count[editor_name] += 1
        else:
            kept_editor_count[editor_name] = 1
    
    # get total number of token added for each editor
    a = fin_df[fin_df['page.title'] == title]
    all_indices = list(a.index)
    
#     response = ww.api.all_content(title)
    try:
        response = ww.api.all_content(title)
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        print(e)
        continue
    all_tokens_added = {}
    all_tokens = response['all_tokens']
    for i in all_tokens: 
        # if the token was added after the most current content we have
        # then do not count it
        if str(i['o_rev_id']) not in all_indices:
            continue
        editor_id = i['editor']
        if editor_id not in contributor_info:
            continue
        editor_name = contributor_info[editor_id]
        if editor_name not in kept_users_list:
            continue
        if editor_name in all_tokens_added:
            all_tokens_added[editor_name] += 1
        else:
            all_tokens_added[editor_name] = 1
    
#     res = {}
#     for i,v in kept_editor_count.items():
#         res[i] = v/all_tokens_added[i]
#     sr[title] = res

    sr[title] = {'all_added':all_tokens_added,'survived':kept_editor_count}
    
    
    ########### end of sr ##################
    
    # hl
    a = a.join(timestamp)
    a = a[~a.index.duplicated(keep='first')]
    a = a.sort_values('revision.timestamp')
    a.insert(0, 'nth_revision', range(0, len(a)))
    
    
    # res stores hl for each rev (per token)
    res = {}

    def add_to_res(ind,val):
        if ind in res:
            res[ind].append(val)
        else:
            res[ind] = [val]

    max_n = a['nth_revision'].max()

    for v in all_tokens:
        if str(v['o_rev_id']) not in all_indices:
            continue
        editor_id = v['editor']
        if editor_id not in contributor_info:
            continue
        editor_name = contributor_info[editor_id]

        # check if editor is kept
        if editor_name not in kept_users_list:
            continue

        # in and out only look at the revisions we have
    #     ins = v['in']
    #     ins = [i for i in ins if str(i) in all_indices]
        outs = v['out']
        outs = [i for i in outs if str(i) in all_indices]

        o_rev = v['o_rev_id']

        if len(outs) == 0:
            age = max_n - a.loc[str(o_rev)]['nth_revision']
            add_to_res(o_rev,age)
            continue

        deleted_n = a.loc[str(outs[-1])]['nth_revision']
        curr_n = a.loc[str(o_rev)]['nth_revision']
        add_to_res(o_rev,deleted_n - curr_n)

    curr_hl = {}
    for i,v in res.items():
        v.sort()
        mid = math.ceil(len(v)/2) - 1
        curr_hl[str(i)] = v[mid]

    hl[title] = curr_hl

Connection failed (attempt 1 of 2) 
Connection failed (attempt 2 of 2) 
400 Client Error: Bad Request for url: https://api.wikiwho.net/en/api/v1.0.0-beta/rev_content/Agriculture%20in%20Chinese%20mythology/503367746/?o_rev_id=true&editor=true&token_id=true&out=true&in=true
Connection failed (attempt 1 of 2) 
Connection failed (attempt 2 of 2) 
400 Client Error: Bad Request for url: https://api.wikiwho.net/en/api/v1.0.0-beta/rev_content/Daoyin/228921533/?o_rev_id=true&editor=true&token_id=true&out=true&in=true
Connection failed (attempt 1 of 2) 
Connection failed (attempt 2 of 2) 
400 Client Error: Bad Request for url: https://api.wikiwho.net/en/api/v1.0.0-beta/rev_content/Ephedra/902098178/?o_rev_id=true&editor=true&token_id=true&out=true&in=true
Connection failed (attempt 1 of 2) 
Connection failed (attempt 2 of 2) 
400 Client Error: Bad Request for url: https://api.wikiwho.net/en/api/v1.0.0-beta/rev_content/Gallbladder%20(Chinese%20medicine)/683162880/?o_rev_id=true&editor=true&token_

In [20]:
all_added_df = pd.concat({
        k: pd.DataFrame.from_dict(v['all_added'], 'index') for k, v in sr.items()
    }, 
    axis=0).reset_index().rename(columns={'level_0':'page.title','level_1':'contributor.username',0:'all_added_tokens'})

In [21]:
survive_df = pd.concat({
        k: pd.DataFrame.from_dict(v['survived'], 'index') for k, v in sr.items()
    }, 
    axis=0).reset_index().rename(columns={'level_0':'page.title','level_1':'contributor.username',0:'survived_tokens'})

In [22]:
all_added_df = all_added_df.set_index(['contributor.username','page.title'])

In [23]:
survive_df = survive_df.set_index(['contributor.username','page.title'])

In [24]:
res = all_added_df.join(survive_df).fillna(0)

In [26]:
res.to_parquet('../../result/TCM/TCM-sr')

In [27]:
fin_hl = pd.concat({
        k: pd.DataFrame.from_dict(v, 'index') for k, v in hl.items()
    }, 
    axis=0).reset_index().rename(columns={'level_0':'page.title','level_1':'index',0:'hl'}).set_index('index')

In [30]:
fin_hl.to_parquet('../../result/TCM/TCM-hl')

In [31]:
# editor's hl

In [32]:
df = fin_hl.drop(columns={'page.title'}).join(fin_df)

In [33]:
df = df.drop(columns={'contributor.id'})

In [35]:
df.to_parquet('../../result/TCM/TCM-hl-with-editor')